In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

embeddings = OpenAIEmbeddings()

loader = UnstructuredFileLoader(
    "./files/chapter_one.txt",
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)


cache_dir = LocalFileStore("./.cache/")

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=cache_dir,
)

vector_store = FAISS.from_documents(docs, cache_embeddings)

retriever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            ------
            {context}
            """,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

map_doc_chain = prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # results = []
    # for document in documents:
    #     result = map_doc_chain.invoke(
    #         {
    #             "context": document.page_content,
    #             "question": question,
    #         }
    #     ).content
    #     results.append(result)
    # results = "\n\n".join(results)
    # return results
    return "\n\n".join(
        map_doc_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        for doc in documents
    )


map_chain = {
    # 자동으로 최종 question이 전달된다
    # 전달된 qustion과 관련된 documents들을 뽑아 내는 것
    "documents": retriever,
    # 자동으로 최종 question이 전달된다
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

chain = (
    {
        "context": map_chain,
        "question": RunnablePassthrough(),
    }
    | final_prompt
    | llm
)

chain.invoke(
    "Where dose Winston go to work?",
)


# """
#     MapReduce Chain Type
#     list of docs :
#     for doc in list of docs \ prompt \ llm
#     for response in list of llms response \ put them all together
#     final doc \ prompt \ llm
# """

AIMessage(content='Winston goes to work at the Ministry of Truth, also known as Minitrue in Newspeak, in George Orwell\'s novel "1984."')